### Classification based on extracted features

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
import numpy as np

Using TensorFlow backend.


In [2]:
model = VGG16(weights='imagenet', include_top=True)

553467904/553467096 [==============================] - 47s 0us/step


In [4]:
features_model = Model(inputs=model.input, outputs=model.layers[-1].output)

In [7]:
test_dir = './data/training_folder/test'
val_dir = './data/training_folder/val'
image_size = (224,224)

In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg


test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg) 
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg) 

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=image_size,
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=image_size,
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)

predict_test = model.predict_generator(test_generator,steps=nb_samples)
predict_val = model.predict_generator(val_generator,steps=nb_samples)

Found 340 images belonging to 17 classes.
Found 340 images belonging to 17 classes.


Now we'll use classical ML algorithm on extracted features

Let us use only the first two classes

In [9]:
features_test = predict_test[test_generator.classes <=1]
features_train = predict_val[val_generator.classes <=1]

y_test = test_generator.classes[test_generator.classes <=1]
y_train = val_generator.classes[val_generator.classes <=1]

In [10]:
from sklearn.linear_model import LogisticRegression

In [11]:
clf = LogisticRegression()

In [12]:
clf.fit(features_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
pred = clf.predict(features_test)

In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(pred, y_test)

0.8